In [1]:
%load_ext autoreload
%autoreload 2
import sys
path='../'
if not path in sys.path:
    
    sys.path.append(path)

### Modeling

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from joblib import dump,load
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import LinearSVC
import numpy as np
import pickle
import pandas as pd

In [3]:
selected_features = pd.read_csv("../data/features.csv")
y = pd.read_csv("../data/label.csv")

In [4]:
len(selected_features)

93225

In [17]:
selected_features = selected_features.drop(columns=['Unnamed: 0'])
selected_features

,discharge_disposition_id,time_in_hospital,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed
0,1,3,18,0,0,0,9,0,1
1,1,2,16,0,0,0,7,0,1
2,1,1,8,0,0,0,5,0,1
3,1,3,16,0,0,0,9,1,1
4,1,5,12,0,0,0,8,1,1
...,...,...,...,...,...,...,...,...,...
93220,3,3,16,0,0,0,9,0,1
93221,4,5,18,0,0,1,9,1,1
93222,1,1,9,1,0,0,13,0,1
93223,3,10,21,0,0,1,9,0,1


In [8]:
y['readmitted'].value_counts()

readmitted
2    50248
1    32620
0    10357
Name: count, dtype: int64

In [18]:
xtrain,xtest,ytrain,ytest = train_test_split(selected_features,y['readmitted'],test_size=0.2,random_state=40)

# creating scaler scale var.
norm = MinMaxScaler()
# fit the scal
norm_fit = norm.fit(xtrain)
## save to pickel file
pickle.dump(norm_fit, open("scaler.pkl", 'wb'))
dump(norm_fit,'scaler.joblib')
# transfromation of trainig data
scal_xtrain = norm_fit.transform(xtrain)

# transformation of testing data
scal_xtest = norm_fit.transform(xtest)
print(scal_xtrain)

[[0.         0.23076923 0.29487179 ... 0.26666667 0.         1.        ]
 [0.         0.15384615 0.05128205 ... 0.53333333 0.         1.        ]
 [0.11111111 0.61538462 0.1025641  ... 0.53333333 1.         0.        ]
 ...
 [0.07407407 0.15384615 0.14102564 ... 0.53333333 0.         1.        ]
 [0.         0.15384615 0.16666667 ... 0.2        0.         1.        ]
 [0.62962963 0.30769231 0.08974359 ... 0.53333333 1.         0.        ]]


In [19]:

# create model variable
rnd = RandomForestClassifier()

# fit the model
fit_rnd = rnd.fit(xtrain,ytrain)

In [8]:
# Perform 5-fold cross-validation
cv_scores = cross_val_score(rnd, xtrain, ytrain, cv=5)

# Print the average score
print(f"Average CV Score: {np.mean(cv_scores):.4f}")

Average CV Score: 0.5485


In [29]:
xtest.iloc[10854].to_dict()

{'discharge_disposition_id': 6,
 'time_in_hospital': 11,
 'num_medications': 18,
 'number_outpatient': 0,
 'number_emergency': 1,
 'number_inpatient': 0,
 'number_diagnoses': 9,
 'change': 0,
 'diabetesMed': 1}

In [32]:
import sklearn
print(sklearn.__version__)

1.2.2


In [31]:
single_instance = pd.DataFrame([xtest.iloc[10854].to_dict()])
prediction = fit_rnd.predict(single_instance)

In [20]:
#predicting our target variable
y_predict = fit_rnd.predict(xtest)

#predicting the probability of our target variable
y_predict_probabilty = fit_rnd.predict_proba(xtest)

In [21]:
print(list(y_predict)[:10])

[2, 2, 2, 2, 2, 1, 2, 2, 1, 2]


In [22]:
print(y_predict_probabilty)

[[0.055      0.13       0.815     ]
 [0.005      0.43166667 0.56333333]
 [0.02416667 0.2465     0.72933333]
 ...
 [0.0922336  0.52965657 0.37810983]
 [0.3        0.41       0.29      ]
 [0.07231252 0.38550382 0.54218366]]


In [23]:
# checking the accuracy score
rnd_score = fit_rnd.score(xtest,ytest)

print('score of model is : ',rnd_score)

pickle.dump(fit_rnd, open("model.pkl", 'wb'))


score of model is :  0.5242156074014481


In [24]:
from joblib import dump

# Assuming your RandomForestClassifier is named 'rnd'
dump(rnd, '../data/random_forest_model.joblib')

['../data/random_forest_model.joblib']

In [25]:
from sklearn.metrics import classification_report
x_predict = list(rnd.predict(xtest))
print(classification_report(ytest, x_predict))
#df = {'predicted':x_predict,'orignal':ytest}
#df

              precision    recall  f1-score   support

           0       0.17      0.08      0.11      2024
           1       0.44      0.37      0.40      6689
           2       0.59      0.72      0.65      9932

    accuracy                           0.52     18645
   macro avg       0.40      0.39      0.39     18645
weighted avg       0.49      0.52      0.50     18645



### Improvement 
### Advanced model logging, registration and data versioning
## DVC and MLflow 



In [34]:
import mlflow
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



# Start an MLflow experiment
with mlflow.start_run():

    # Train a model
    model = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    model.fit(xtrain,ytrain)

    # Make predictions
    predictions = model.predict(xtest)

    # Log an evaluation metric
    accuracy = accuracy_score(ytest, predictions)
    mlflow.log_metric("accuracy", accuracy)

    print("Model accuracy: %f" % accuracy)


2023/12/13 15:04:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mahlettaye/miniconda3/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
/Users/mahlettaye/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is

Model accuracy: 0.568839
